In [1]:
import pandas as pd
import networkx as nx
import numpy as np

In [2]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'

In [3]:
stop_times_reduced = pd.read_csv(gtfs_dir + 'stop_times_reduced.csv')
stop_times_reduced

,trip_id,stop_sequence_new,stop_sequence,hr,min,INT_ID
0,32828689,1,1,5,15,13466640.0
1,32828689,2,2,5,16,13466531.0
2,32828689,3,3,5,16,13466677.0
3,32828689,4,4,5,17,13466754.0
4,32828689,5,5,5,18,13466699.0
...,...,...,...,...,...,...
3959705,32970576,7,8,25,56,13461602.0
3959706,32970576,8,9,25,57,13461912.0
3959707,32970576,9,10,25,57,13462284.0
3959708,32970576,10,11,25,58,13462427.0


In [4]:
hr_list = [28,29,6,7,8]
departure_hour = [28]

In [5]:
stop_times_reduced = stop_times_reduced[stop_times_reduced['hr'].isin(hr_list)]

stop_times_reduced['hr'] = np.where(stop_times_reduced['hr']>23, stop_times_reduced['hr'] - 24, stop_times_reduced['hr'])


/Users/Rick/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
stop_times_reduced_o = stop_times_reduced.copy()
stop_times_reduced_d = stop_times_reduced.copy()
stop_times_reduced_d['stop_sequence_new'] = stop_times_reduced_d['stop_sequence_new'] - 1

In [7]:
stop_times_od = stop_times_reduced_o.merge(stop_times_reduced_d, left_on = ['trip_id', 'stop_sequence_new'],
                           right_on = ['trip_id', 'stop_sequence_new'], suffixes = ['_o', '_d'])

In [8]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d
0,32828690,28,31,6,0,13464667.0,32,6,1,13464586.0
1,32828690,29,32,6,1,13464586.0,33,6,1,13464490.0
2,32828690,30,33,6,1,13464490.0,34,6,2,13464269.0
3,32828690,31,34,6,2,13464269.0,35,6,3,13464177.0
4,32828690,32,35,6,3,13464177.0,36,6,4,13464004.0
...,...,...,...,...,...,...,...,...,...,...
537660,32970559,50,54,5,25,13466699.0,55,5,26,13466754.0
537661,32970559,51,55,5,26,13466754.0,56,5,27,13466677.0
537662,32970559,52,56,5,27,13466677.0,57,5,27,13466531.0
537663,32970559,53,57,5,27,13466531.0,58,5,28,13466258.0


In [9]:
stop_times_od['min_o'] = stop_times_od['min_o'].apply(lambda x: '{0:0>2}'.format(x))
stop_times_od['min_d'] = stop_times_od['min_d'].apply(lambda x: '{0:0>2}'.format(x))

In [10]:
stop_times_od['hr_o'] = stop_times_od['hr_o'].astype(str)
stop_times_od['hr_d'] = stop_times_od['hr_d'].astype(str)
# stop_times_od['min_o'] = stop_times_od['min_o'].astype(str)

In [11]:
stop_times_od['o_time_str'] = stop_times_od['hr_o'] + ':' + stop_times_od['min_o']
stop_times_od['d_time_str'] = stop_times_od['hr_d'] + ':' + stop_times_od['min_d']

In [12]:
stop_times_od['o_time'] = pd.to_datetime(stop_times_od['o_time_str'], format = '%H:%M')
stop_times_od['d_time'] = pd.to_datetime(stop_times_od['d_time_str'], format = '%H:%M')


In [13]:
stop_times_od['cost'] = ((stop_times_od['d_time'] - stop_times_od['o_time']).dt.seconds)/60

In [14]:
stop_times_od['o_time'] = stop_times_od['o_time'].dt.time
stop_times_od['d_time'] = stop_times_od['d_time'].dt.time

In [15]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,o_time_str,d_time_str,o_time,d_time,cost
0,32828690,28,31,6,00,13464667.0,32,6,01,13464586.0,6:00,6:01,06:00:00,06:01:00,1.0
1,32828690,29,32,6,01,13464586.0,33,6,01,13464490.0,6:01,6:01,06:01:00,06:01:00,0.0
2,32828690,30,33,6,01,13464490.0,34,6,02,13464269.0,6:01,6:02,06:01:00,06:02:00,1.0
3,32828690,31,34,6,02,13464269.0,35,6,03,13464177.0,6:02,6:03,06:02:00,06:03:00,1.0
4,32828690,32,35,6,03,13464177.0,36,6,04,13464004.0,6:03,6:04,06:03:00,06:04:00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537660,32970559,50,54,5,25,13466699.0,55,5,26,13466754.0,5:25,5:26,05:25:00,05:26:00,1.0
537661,32970559,51,55,5,26,13466754.0,56,5,27,13466677.0,5:26,5:27,05:26:00,05:27:00,1.0
537662,32970559,52,56,5,27,13466677.0,57,5,27,13466531.0,5:27,5:27,05:27:00,05:27:00,0.0
537663,32970559,53,57,5,27,13466531.0,58,5,28,13466258.0,5:27,5:28,05:27:00,05:28:00,1.0


In [16]:
trips_weekday = pd.read_csv(gtfs_dir + 'trips_weekday.csv')

In [17]:
stop_times_od = stop_times_od.merge(trips_weekday[['trip_id', 'route_short_name', 'direction_id']], how = 'inner')

In [18]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,o_time_str,d_time_str,o_time,d_time,cost,route_short_name,direction_id
0,32828690,28,31,6,00,13464667.0,32,6,01,13464586.0,6:00,6:01,06:00:00,06:01:00,1.0,506,0
1,32828690,29,32,6,01,13464586.0,33,6,01,13464490.0,6:01,6:01,06:01:00,06:01:00,0.0,506,0
2,32828690,30,33,6,01,13464490.0,34,6,02,13464269.0,6:01,6:02,06:01:00,06:02:00,1.0,506,0
3,32828690,31,34,6,02,13464269.0,35,6,03,13464177.0,6:02,6:03,06:02:00,06:03:00,1.0,506,0
4,32828690,32,35,6,03,13464177.0,36,6,04,13464004.0,6:03,6:04,06:03:00,06:04:00,1.0,506,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228704,32970469,6,7,5,49,13461421.0,8,5,49,13461602.0,5:49,5:49,05:49:00,05:49:00,0.0,306,1
228705,32970469,7,8,5,49,13461602.0,9,5,50,13461912.0,5:49,5:50,05:49:00,05:50:00,1.0,306,1
228706,32970469,8,9,5,50,13461912.0,10,5,50,13462284.0,5:50,5:50,05:50:00,05:50:00,0.0,306,1
228707,32970469,9,10,5,50,13462284.0,11,5,51,13462427.0,5:50,5:51,05:50:00,05:51:00,1.0,306,1


Inner merge because some trips in the stop times files have `service_id` 2 and 3 (meaning they run on weekends).

Type 1 nodes are boarding, while type 2 nodes are alighting. Type 0 nodes are regular waiting nodes. This is to penalize transfers, and make transfering more realistic. Links connecting type 1 and type 2 nodes to type 0 nodes will be added. Type 3 nodes are destination constrained nodes (to distinguish from origin zones)

In [19]:
stop_times_od['node_o'] = stop_times_od['INT_ID_o'].astype(
    int).astype(str) + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-'+ stop_times_od['hr_o'].astype(
    str) + '-' + stop_times_od['min_o'].astype(int).astype(str)
stop_times_od['node_d'] = stop_times_od['INT_ID_d'].astype(
    int).astype(str) + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-'+ stop_times_od['hr_d'].astype(
    str) + '-' + stop_times_od['min_d'].astype(int).astype(str)

In [20]:
stop_times_od['link'] = '0' + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-' + stop_times_od['trip_id'].astype(str) + '-' + stop_times_od['direction_id'].astype(str)+ '_' + stop_times_od['node_o'] + '_' + stop_times_od['node_d']

In [21]:
stop_times_od['cost'] = stop_times_od['cost'].astype(int)

In [22]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,o_time_str,d_time_str,o_time,d_time,cost,route_short_name,direction_id,node_o,node_d,link
0,32828690,28,31,6,00,13464667.0,32,6,01,13464586.0,6:00,6:01,06:00:00,06:01:00,1,506,0,13464667-506-6-0,13464586-506-6-1,0-506-32828690-0_13464667-506-6-0_13464586-506...
1,32828690,29,32,6,01,13464586.0,33,6,01,13464490.0,6:01,6:01,06:01:00,06:01:00,0,506,0,13464586-506-6-1,13464490-506-6-1,0-506-32828690-0_13464586-506-6-1_13464490-506...
2,32828690,30,33,6,01,13464490.0,34,6,02,13464269.0,6:01,6:02,06:01:00,06:02:00,1,506,0,13464490-506-6-1,13464269-506-6-2,0-506-32828690-0_13464490-506-6-1_13464269-506...
3,32828690,31,34,6,02,13464269.0,35,6,03,13464177.0,6:02,6:03,06:02:00,06:03:00,1,506,0,13464269-506-6-2,13464177-506-6-3,0-506-32828690-0_13464269-506-6-2_13464177-506...
4,32828690,32,35,6,03,13464177.0,36,6,04,13464004.0,6:03,6:04,06:03:00,06:04:00,1,506,0,13464177-506-6-3,13464004-506-6-4,0-506-32828690-0_13464177-506-6-3_13464004-506...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228704,32970469,6,7,5,49,13461421.0,8,5,49,13461602.0,5:49,5:49,05:49:00,05:49:00,0,306,1,13461421-306-5-49,13461602-306-5-49,0-306-32970469-1_13461421-306-5-49_13461602-30...
228705,32970469,7,8,5,49,13461602.0,9,5,50,13461912.0,5:49,5:50,05:49:00,05:50:00,1,306,1,13461602-306-5-49,13461912-306-5-50,0-306-32970469-1_13461602-306-5-49_13461912-30...
228706,32970469,8,9,5,50,13461912.0,10,5,50,13462284.0,5:50,5:50,05:50:00,05:50:00,0,306,1,13461912-306-5-50,13462284-306-5-50,0-306-32970469-1_13461912-306-5-50_13462284-30...
228707,32970469,9,10,5,50,13462284.0,11,5,51,13462427.0,5:50,5:51,05:50:00,05:51:00,1,306,1,13462284-306-5-50,13462427-306-5-51,0-306-32970469-1_13462284-306-5-50_13462427-30...


In [23]:
walk_links = pd.read_csv('GIS/int_tts_walk_time.csv')
walk_links['duration'] = round(walk_links['duration']/60).astype(int)
walk_links['gta06'] = walk_links['gta06'] + 1000
walk_links

,gta06,INT_ID,duration
0,1001,13461602,25
1,1001,13461733,21
2,1001,13462844,20
3,1001,13461863,27
4,1001,13463408,9
...,...,...,...
37157,1081,13467412,24
37158,1081,13467854,6
37159,1081,13466312,22
37160,1081,13467197,29


In [24]:
G = nx.DiGraph()

## Adding node-times

In [25]:
nodes = list((stop_times_od['INT_ID_o'].append(stop_times_od['INT_ID_d'])).drop_duplicates())

In [26]:
node_list = []
for node in nodes:
    for hr_i in hr_list:
        if hr_i > 23:  
            hr = hr_i - 24
        else:
            hr = hr_i
        for minute in range(60):
            node_id = str(int(node)) + '-0-' + str(hr) + '-' + str(minute)
            node_list.append(node_id)

In [27]:
G.add_nodes_from(node_list)

## Adding Boarding Nodes

In [28]:
boarding_nodes = list(stop_times_od['node_o'].drop_duplicates())

In [29]:
G.add_nodes_from(boarding_nodes)

## Adding Alighting Nodes

In [30]:
alighting_nodes = list(stop_times_od['node_d'].drop_duplicates())

In [31]:
G.add_nodes_from(alighting_nodes)

## Adding travel links

In [32]:
link_list = []
for index, row in stop_times_od.iterrows():
    attributes = {'cost': row['cost']}
    link = (row['node_o'], row['node_d'], attributes)
    link_list.append(link)

In [33]:
G.add_edges_from(link_list)

## Adding waiting links

In [34]:
transfer_list = []
for node in nodes:
    next_hr = 4
    next_minute = 0
    node = int(node)
    for hr_i in hr_list:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
            if current_minute == 59:
                next_minute = 0
                next_hr = current_hr + 1
                
            else:
                next_minute = current_minute + 1
            node_o = str(node) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(node) + '-0-' + str(next_hr) + '-' + str(next_minute)
            
            link_name = '1-0-0-9_' + node_o + '_' + node_d
            
            attributes = {'cost': 1}
            
            
            link = (node_o, node_d, attributes)
            transfer_list.append(link)
    

In [35]:
G.add_edges_from(transfer_list)

## Adding Boarding Links

As a start, we'll match Google Maps minimum transfer time of 4 minutes, with 2 minute cost for both boarding and alighting links, meaning transfers be a minimum of 4 minutes.

A direction id of 1 means inbound to the stop node.

In [36]:
boarding_link = []
for node in boarding_nodes:
    node_split = node.split('-')
    int_id = node_split[0]
    boarding_hr = int(node_split[2])
    boarding_minute = int(node_split[3])

    stop_minute = boarding_minute - 2
    if stop_minute < 0:
        stop_hr = boarding_hr -1
        stop_minute = 60 + stop_minute
    else:
        stop_hr = boarding_hr

    stop_node = int_id + '-0-' + str(stop_hr) + '-' + str(stop_minute)
    attributes = {'cost': 2}
    
    link = (stop_node, node, attributes)
    
    boarding_link.append(link)

In [37]:
G.add_edges_from(boarding_link)

## Adding Alighting Links

In [38]:
alighting_link = []
for node in alighting_nodes:
    node_split = node.split('-')
    int_id = node_split[0]
    alighting_hr = int(node_split[2])
    alighting_minute = int(node_split[3])

    stop_minute = alighting_minute + 2
    if stop_minute > 59:
        stop_hr = alighting_hr + 1
        stop_minute = stop_minute - 60
    else:
        stop_hr = alighting_hr

    stop_node = int_id + '-0-' + str(stop_hr) + '-' + str(stop_minute)
    attributes = {'cost': 2}
    
    link = (node, stop_node, attributes)
    
    alighting_link.append(link)

In [39]:
G.add_edges_from(alighting_link)

## Adding TTS Zone Origins

Will also follow node-time structure

In [40]:
tts_zones = list(walk_links['gta06'].drop_duplicates())

In [41]:
node_stationary_list = []
for zone in tts_zones:
    for hr_i in hr_list:
        if hr_i > 23:  
            hr = hr_i - 24
        else:
            hr = hr_i
        for minute in range(60):
            node_id = str(int(zone)) + '-0-' + str(hr) + '-' + str(minute)
            node_stationary_list.append(node_id)

In [42]:
G.add_nodes_from(node_stationary_list)

## Adding TTS Zone Destinations

Marked by type 3 nodes, will also follow node-time structure

In [43]:
node_dest_list = []
for zone in tts_zones:
    for hr_i in hr_list:
        if hr_i > 23:  
            hr = hr_i - 24
        else:
            hr = hr_i
        for minute in range(60):
            node_id = str(int(zone)) + '-3-' + str(hr) + '-' + str(minute)
            node_dest_list.append(node_id)

In [44]:
G.add_nodes_from(node_dest_list)

## Adding links from TTS to intersections

Both outbound and inbound links will be added. In pathfinding process, only inbound links for the destination, and outbound links for the origin will be kept.

Only links during the departure hour will be made due to space concerns.

Convention is 0 for outbound and 1 for inbound

In [45]:
walk_links.head()

,gta06,INT_ID,duration
0,1001,13461602,25
1,1001,13461733,21
2,1001,13462844,20
3,1001,13461863,27
4,1001,13463408,9


In [46]:
walk_link_outbound = []
for index, row in walk_links.iterrows():
    
    zone = int(row['gta06'])
    node = int(row['INT_ID'])
    cost = int(row['duration'])
    
    for hr_i in departure_hour:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
            next_hr = current_hr
            next_minute = current_minute + cost
            if next_minute > 59:
                next_hr= current_hr + 1
                next_minute = next_minute - 60
                
            node_o = str(zone) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(node) + '-0-' + str(next_hr) + '-' + str(next_minute)
            
            link_name = '2-0-0-0_' + node_o + '_' + node_d
            
            attributes = {'cost': cost}
    
            link = (node_o, node_d, attributes)
            walk_link_outbound.append(link)

## Adding links from intersections to TTS Zones

In [47]:
walk_link_inbound = []
for index, row in walk_links.iterrows():
    
    zone = int(row['gta06'])
    node = int(row['INT_ID'])
    cost = int(row['duration'])
    
    for hr_i in hr_list:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
            next_hr = current_hr
            next_minute = current_minute + cost
            if next_minute > 59:
                next_hr= current_hr + 1
                next_minute = next_minute - 60
                
            node_o = str(node) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(zone) + '-9999-' + str(next_hr) + '-' + str(next_minute)
            
            link_name = '2-0-0-1_' + node_o + '_' + node_d
            
            attributes = {'cost': cost}
    
            link = (node_o, node_d, attributes)
            walk_link_inbound.append(link)

In [48]:
G.add_edges_from(walk_link_outbound)

In [49]:
G.add_edges_from(walk_link_inbound)

In [50]:
nx.write_graphml(G,'networks/EM-TE-16-5-constrained.graphml')
stop_times_od.to_csv('networks/EM-TE-16-5-constrained.csv', index = False)

In [51]:
G.number_of_nodes()

1862642

In [52]:
G.number_of_edges()

15084798